In [ ]:
# Scrape information from the website (fawsl = day_game, datetime_game, home_team, away_team, location)
import lxml
import bs4
import requests
url = 'https://womenscompetitions.thefa.com/Fixtures-Results'
result = requests.get(url)
soup = bs4.BeautifulSoup(result.text, 'lxml')
all_games = soup.select('.individual-row.kickoff')
season_info = soup.select('option')

lst_of_games = []
for n in range(len(all_games)):
    game = all_games[n]
    game_list = game.contents
    for i in [3,5,7,9,11]:
        if i == 3:
            day_tag = game_list[3].find('span')
            day_text = day_tag.getText()
            day_str = day_text[:3]
        elif i == 5:
            loc_tag = game_list[5].select('.location')[0]
            loc_str = loc_tag.getText()
            date_time_tag = game_list[5].select('.scInputMatchDate')[0]
            date_time_str = date_time_tag.attrs['value']
        elif i == 7:
            team1_tag = game_list[7].select('.team-name-text')[0]
            team1_str = team1_tag.getText().strip()
        elif i == 11:
            team2_tag = game_list[11].select('.team-name-text')[0]
            team2_str = team2_tag.getText().strip()
    lst_of_games.append([day_str, date_time_str, team1_str, team2_str, loc_str])

In [6]:
# Transport scraped information into a postgreSQL database

# Connect to postgreSQL
import psycopg2 as pg2
conn = pg2.connect(database = 'postgres', user = 'postgres', password = 'conf')
cur = conn.cursor()
conn.autocommit = True

# Create database
create_db = '''CREATE DATABASE fawsl_2324;'''
cur.execute(create_db)

# Change database
conn = pg2.connect(database = 'fawsl_2324', user = 'postgres', password = 'conf')
cur = conn.cursor()
conn.autocommit = True

# Create fawsl table
create_fawsl = '''CREATE TABLE fawsl_2324(
                game_id SERIAL PRIMARY KEY,
                day_game VARCHAR, 
                datetime_game TIMESTAMP, 
                home_team VARCHAR, 
                away_team VARCHAR, 
                location VARCHAR);'''
cur.execute(create_fawsl)

insert_fawsl = '''INSERT into fawsl_2324(
                day_game, 
                datetime_game, 
                home_team, 
                away_team, 
                location)
                VALUES
                (%s, %s, %s, %s, %s);'''

for game in lst_of_games:
    cur.execute(insert_fawsl, game)
    
conn.commit()

# Return the whole fawsl table 
full_table = '''SELECT * FROM fawsl_2324;'''
cur.execute(full_table)
table = cur.fetchall()

In [ ]:
# Connecting to the Google Calendar API
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']

def main():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

# Create fawsl events on Google Calendar from PostgreSQL table of games
    try:
        service = build('calendar', 'v3', credentials=creds)
        
        for each_game in table:
            summary = each_game[3] + ' vs. ' + each_game[4]            
            start = each_game[2]
            start_time = start.isoformat() + 'Z'
            end = start + datetime.timedelta(hours=2)
            end_time = end.isoformat() + 'Z'
            location = each_game[-1]
            
            event = {
            'summary': summary,
            'description': 'https://faplayer.thefa.com/' + '\n' + 'https://www.paramountplus.com/',
            'location': location,
            'start': {'dateTime': start_time, 'timeZone':'America/Chicago'},
            'end': {'dateTime': end_time, 'timeZone':'America/Chicago'}
            }
            
            event = service.events().insert(calendarId='primary', body=event).execute()
    
    except HttpError as error:
        print('An error occurred: %s' % error)

if __name__ == '__main__':
    main()